# 11-1 단어의 표현 방법

## 1. 단어의 표현 방법
국소 표현(Local Representation): 해당 단어 그 자체만 보고, 특정값을 맵핑하여 단어를 표현하는 방법 
> puppy, cute, lovely라는 단어 각각에 1번, 2번 3번 등과 같은 숫자를 mapping하여 부여  
 
분산 표현(Distributed Representation): 그 단어를 표현하고자 주변을 참고하여 단어를 표현
> puppy라는 단어 근처에는 주로 cute, lovely라는 단어가 자주 등장하므로, puppy라는 단어는 cute, lovely한 느낌이다로 단어 정의

## 2. 단어 표현의 카테고리화
<p align="center">
    <img src="https://wikidocs.net/images/page/31767/wordrepresentation.PNG" >
</p>

# 11-1 Bag of Words

**Bag of Words**: 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
> [과정]  
> (1) 각 단어에 고유한 정수 인덱스를 부여  
> (2) 각 index의 위치에 단어 트큰의 등장 횟수를 기록한 벡터를 만듦

**문서 1: 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.**  

위 문서 1에 대해 BoW를 만들어봄

In [2]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # BoW에 전부 기본값 1을 넣음
            bow.insert(len(word_to_index)-1, 1)
        else:
            # 재등장하는 단어의 index
            index = word_to_index[word]
            bow[index] += 1

    return word_to_index, bow



In [3]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

vocab, bow = build_bag_of_words(doc1)
print('vocabulary:', vocab)
print('bag of words vector:', bow)

vocabulary: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


## 2. BoW의 다른 예제들

**문서 2: 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.**

In [4]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


## 3. CountVectorizer 클래스로 BoW 만들기

Scikit-Learn에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원  
(영어에서 손쉽게 BoW를 만들 수 있음)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector:', vector.fit_transform(corpus).toarray())

# 각 단어의 index가 어떻게 부여되었는지 출력
print('vocabulary:', vector.vocabulary_)

bag of words vector: [[1 1 2 1 2 1]]
vocabulary: {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


I는 BoW를 만드는 과정에서 사라졌는데, 이는 CountVectorizer가 기본적으로 길이가 2 이상인 문자에 대해서만 토큰으로 인식하기 때문에 사라짐  

- 주의  
  CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 잘라 한국어에서 사용하기 힘들다  

## 4. 불용어를 제거한 BoW 만들기

CountVectorizer 는 불용어를 지정하면, 불용어는 제외하고 BoW를 만들 수 있도록 함

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

### (1) 사용자가 직접 정의한 불용어 사용

In [12]:
text = ["Family is not an important thing. It's everything."]

vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])

print('bag of words vector:', vect.fit_transform(text).toarray())
print('vocabulary:', vect.vocabulary_)

bag of words vector: [[1 1 1 1 1]]
vocabulary: {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


### (2) CountVectorizer에서 제공하는 자체 불용어 사용

In [14]:
text = ["Family is not an important thing. It's everything."]

vect = CountVectorizer(stop_words="english")

print('bag of words vector:', vect.fit_transform(text).toarray())
print('vocabulary:', vect.vocabulary_)

bag of words vector: [[1 1 1]]
vocabulary: {'family': 0, 'important': 1, 'thing': 2}


### (3) NLTK에서 지원하는 불용어 사용

In [15]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")

vect = CountVectorizer(stop_words=stop_words)

print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 11-3 문서의 벡터화: 문서 단어 행렬(Document-Term Matrix)

문서 단어 행렬(Document-Term Matrix, DTM): 서로 다른 문서들의 BoW들을 결합한 표현 방법

## 1. 문서 단어 행렬의 표기법

문서 단어 행렬: 다수의 문서에 등장하는 각 단어들의 빈도를 행렬로 표현한 것  
즉, 각 문서에 대한 BoW를 하나의 행렬로 만든 것  

>[예시]  
>문서1 : 먹고 싶은 사과  
>문서2 : 먹고 싶은 바나나  
>문서3 : 길고 노란 바나나 바나나  
>문서4 : 저는 과일이 좋아요  

띄어쓰기 단위 tokenization이라 가정하면
||과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|---|---|---|---|---|---|---|---|---|---|
|문서1|0|0|0|1|0|1|1|0|0|
|문서2|0|0|0|1|1|0|1|0|0|
|문서3|0|1|1|0|2|0|0|0|0|
|문서4|1|0|0|0|0|0|0|1|1|

==> 서로 비교할 수 있도록 수치화 할 수 있다는 점에서 의의를 가짐

## 2. 문서 단어 행렬의 한계

### 1) Sparse Representation
DTM은 각 행을 문서 벡터라 가정하면 각 문서의 차원은 전체 단어 집합의 크기를 가짐  
즉, corpus가 방대한 데이터라면 문서 벡터의 차원이 매우 커짐  
또한 많은 문서의 대부분 값은 0을 가질 것이고 원-핫 벡터와 같이 공간적 낭비, 계산 리소스 증가의 단점

이 대부분 값이 0인 표현을 sparse vector라고 하는데 이는 많은 양의 저장 공간과 높은 계산 복잡도를 요구하므로 전처리를 통해 단어 집합의 크기를 줄이는 일이 중요  

### 2) 단순 빈도 수 기반 접근
모든 단어에 대해 빈도 표기를 하는 방법은 다음의 한계를 가짐

불용어 the의 경우 어떤 문서이든 자주 등장하는데 문서1, 문서2, 문서3에서 동일하고 the 빈도수가 높다고 해서 이 문서들이 유사한 문서라고 판단해서는 안됨  
따라서 DTM에서 불용어와 중요한 단어에 대해 가중치를 줄 수 있는 TF-IDF를 다음에 소개함

# 11-4 TF-IDF(Term Frequency-Inverse Document Frequency)

## 1. TF-IDF

TF-IDF: 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용해 DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방법  
우선 DTM 생성 -> TF-IDF 가중치 부여  

$$TF-IDF=TF \times IDF$$

$d: 문서, t: 단어, n: 문서 총 개수$

### (1) tf(d, t): 특정 문서 d에서의 특정 단어 t의 등장 횟수
DTM에서 각 단어들이 가진 값들

### (2) df(t): 특정 단어 t가 등장한 문서의 수

### (3) idf(t): df(t)에 반비례하는 수

$$idf(t)=log(\frac{n}{1+df(t)})$$

[**log를 사용하는 이유 1**]  
문서의 수 n이 커질 수록, IDF의 값은 기하급수적으로 커지게 되는데 이를 방지하기 위해

[예시]
n = 1,000,000일 때, log의 밑은 10을 사용  
|단어$t$|$df(t)$|$idf(d,t)$|
|---|---|---|
|word1|1|6|
|word2|100|4|
|word3|1,000|3|
|word4|10,000|2|
|word5|100,000|1|
|word6|1,000,000|0|

log를 사용하지 않았을 때
$idf(t)=n/df(t)$
|단어$t$|$df(t)$|$idf(t)$|
|---|---|---|
|word1|1|1,000,000|
|word2|100|10,000|
|word3|1,000|1,000|
|word4|10,000|100|
|word5|100,000|10|
|word6|1,000,000|1|

[**log를 사용하는 이유 2**]  
불용어 등 자주 쓰이는 단어들은 자주 쓰이지 않는 단어들보다 최소 수십 배 자주 등장  
이는 수백 배 더 자주 등장할 수도 있는데, 이런 경우 log를 씌어주지 않으면, 희귀 단어들에  
엄청난 가중치가 부여됨

**TF-IDF**  
TF-IDF는 모든 문서에 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단  

[예시]  
||과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|---|---|---|---|---|---|---|---|---|---|
|문서1|0|0|0|1|0|1|1|0|0|
|문서2|0|0|0|1|1|0|1|0|0|
|문서3|0|1|1|0|2|0|0|0|0|
|문서4|1|0|0|0|0|0|0|1|1|

TF는 앞서 사용한 DTM을 그대로 사용하면, 그것이 각 문서에서의 각 단어의 TF가 됨  
이제는 IDF를 구해야 하는데, 로그는 자연 로그를 사용하여 계산  

|단어|IDF(역 문서 빈도)|
|---|---|
|과일이|ln(4/(1+1))=0.693147|
|길고|ln(4/(1+1))=0.693147|
|노란|ln(4/(1+1))=0.693147|
|먹고|ln(4/(2+1))=0.287682|
|바나나|ln(4/(2+1))=0.287682|
|사과|ln(4/(1+1))=0.693147|
|싶은|ln(4/(2+1))=0.287682|
|저는|ln(4/(1+1))=0.693147|
|좋아요|ln(4/(1+1))=0.693147|

문서의 총 수는 4이므로 분자는 항상 4로 동일함  
TF-IDF는 위에서 구한 TF값과 IDF를 곱해 구하면 되므로 생략함  

## 2. 파이썬으로 TF-IDF 직접 구현하기

In [16]:
import pandas as pd
from math import log    # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [22]:
N = len(docs)       # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t,d) * idf(t)

In [20]:
# TF를 구함(DTM을 데이터프레임에 저장해 출력)
result = []

# 각 문서에 대해 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]

    for j in range(len(vocab)):
        t = vocab[j]
        result[i].append(tf(t,d))

tf_ = pd.DataFrame(result, columns=vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [23]:
# 각 단어에 대한 IDF 값 구하기
result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [24]:
# TF-IDF 행렬 출력하기
result = []

for i in range(N):
    result.append([])
    d = docs[i]

    for j in range(len(vocab)):
        t = vocab[j]
        result[i].append(tfidf(t, d))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## 3. 사이킷런을 이용한 DTM과 TF-IDF 실습

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 mapping된 index 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


사이킷런은  TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공  

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 11-5 코사인 유사도를 이용한 추천 시스템

## 1. Cosine Similarity
두 벡터 간의 코사인 각도를 이용해 구할 수 있는 두 벡터의 유사도를 의미(-1 ~ 1)  
> 1 : 벡터 방향이 완전히 동일  
> 0 : 90도의 각을 이룸  
> -1: 180도로 완전히 반대의 방향을 가짐  

<p align="center"><img src=https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG></p>

코사인 유사도의 식은 다음과 같음  
$$similarity=cos(\Theta)=\frac{A\cdot B}{||A||||B||}=\frac{\Sigma^n_{i=1}A_i\times B_i}{\sqrt{\Sigma^n_{i=1}(A_i)^2}\times\sqrt{\Sigma^n_{i=1}(B_i)^2}}$$

[예시]  
문서1 : 저는 사과 좋아요  
문서2 : 저는 바나나 좋아요  
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요  
  
띄어쓰기 기준 tokenizatioin이라 가정하고, 세 문서에 대해 DTM은 다음과 같음  
||바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|0|1|1|1|
|문서2|1|0|1|1|
|문서3|2|0|2|2|

- Numpy를 사용해 코사인 유사도를 계산하는 함수 구현

In [32]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

docs = ["저는 사과 좋아요",
        "저는 바나나 좋아요",
        "저는 바나나 좋아요 저는 바나나 좋아요"]

vectorizer = CountVectorizer()
DTM = vectorizer.fit_transform(docs).toarray()

doc1, doc2, doc3 = DTM[0], DTM[1], DTM[2]

print('문서1, 문서2 유사도:', cos_sim(doc1, doc2))
print('문서1, 문서3 유사도:', cos_sim(doc1, doc3))
print('문서2, 문서3 유사도:', cos_sim(doc2, doc3))

문서1, 문서2 유사도: 0.6666666666666667
문서1, 문서3 유사도: 0.6666666666666667
문서2, 문서3 유사도: 1.0000000000000002


## 2. 유사도를 이용한 추천 시스템 구현하기
캐글에서 사용되었던 영화 Dataset을 가지고 영화 추천 시스템 만들기  
TF-IDF, Cosine-Similarity 만으로 영화 줄거리에 기반해 영화를 추천하는 시스탬  

[Dataset]  
https://www.kaggle.com/rounakbanik/the-movies-dataset   
  
총 24개의 열을 가진 45,466개의 sample로 구성된 영화 정보 데이터

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv("movies_metadata.csv")
data.head(2)

C:\Users\gkwjd\AppData\Local\Temp\ipykernel_37916\3967704491.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [34]:
# 상위 2만개의 sample을 data에 저장
data = data.head(20000)

In [37]:
# TF-IDF를 연산할 때 데이터에 Null 값이 들어있으면 에러가 발생하므로 Null값 확인
# overview 열에 존재하는 모든 결측값을 전부 카운트해 출력
print('overview 열의 결측값 수:', data['overview'].isnull().sum())

overview 열의 결측값 수: 135


In [38]:
# Null대신 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

In [40]:
# TF-IDF 행렬 구하기
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape):', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape): (20000, 47487)


In [43]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과:', cosine_sim.shape)

코사인 유사도 연산 결과: (20000, 20000)


20000 X 20000 행렬이 나오는데 이는 20000개의 각 문서 벡터와 자기 자신을 포함한 20000개의 문서 벡터간의 유사도가 기록된 행렬  

In [45]:
# 영화의 타이틀을 key, 영화의 index를 value로 하는 딕셔너리 생성
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father fo the Brige Part II의 index 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [46]:
# 선택한 영화의 제목을 입력해 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 index를 받아옴
    idx = title_to_index[title]

    # 해당 영화와 모든 영화의 유사도를 가져옴
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 index를 얻음
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화 제목을 return
    return data['title'].loc[movie_indices]

In [47]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

# 11-6 단어와 문서의 유사도를 구하는 다양한 방법

## 1. Euclidean distance
다차원 공간에서 두 개의 점 $p$, $q$가 각각 $p=(p_1, p_2, p_3, ..., p_n)$과 $q=(q_1, q_2, q-3, ..., q_n)$의 좌표를 가질 때 두 점 사이의 거리를 계산하는 유클리드 공식은 다음과 같음  
$$\sqrt{(q_1-p_1)^2+(q_2-p_2)^2+...+(q_n-p_n)^2}=\sqrt{\Sigma^n_{i=1}(q_i-p_i)^2}$$

[예시]  
||바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|2|3|0|1|
|문서2|1|2|3|1|
|문서3|2|1|2|2|

단어의 개수가 4개이므로, 4차원 공간에 문서1, 문서2, 문서3을 배치하는 것과 동일  
이 때 다음과 같은 문서 Q에 대해 문서1, 문서2, 문서3 중 가장 유사한 문서를 찾아내고자 함  

||바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서Q|1|1|0|1|

이 때 Euclidean Distance를 통해 휴사도를 구하려고 한다면, 문서 Q또한 다른 문서들처럼 4차원 공간에 배치시켰다는 관점에서 구해야함

In [48]:
import numpy as np

def dist(x, y):
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2, 3, 0, 1))
doc2 = np.array((1, 2, 3, 1))
doc3 = np.array((2, 1, 2, 2))
docQ = np.array((1, 1, 0, 1))

print('문서1과 문서Q의 거리:', dist(doc1, docQ))
print('문서2와 문서Q의 거리:', dist(doc2, docQ))
print('문서3과 문서Q의 거리:', dist(doc3, docQ))

문서1과 문서Q의 거리: 2.23606797749979
문서2와 문서Q의 거리: 3.1622776601683795
문서3과 문서Q의 거리: 2.449489742783178


문서1이 문서Q와 가장 유사하다고 볼 수 있음

## 2. Jaccard Similarity

A와 B 두 개의 집합이 있다고 가정했을 때  
합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있음 -> Jaccard Similarity  
> 1 : 두 집합이 동일함  
> 2 : 공통 원소가 없음

$$J(A,B)=\frac{|A\cap B|}{|A\cup B|}=\frac{|A\cap B|}{|A|+|B|-|A\cap B|}$$

In [49]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [50]:
# 문서1과 문서2의 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서1과 문서2의 합집합:', union)

문서1과 문서2의 합집합: {'likey', 'love', 'like', 'holder', 'coupon', 'card', 'passport', 'you', 'watch', 'everyone', 'banana', 'apple'}


In [51]:
# 문서1과 문서2의 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print('문서1과 문서2의 교집합:', intersection)

문서1과 문서2의 교집합: {'banana', 'apple'}


In [52]:
print('자카드 유사도:', len(intersection)/len(union))

자카드 유사도: 0.16666666666666666
